<a href="https://colab.research.google.com/github/charlessantarosa/dl-emotion-recognition-uea/blob/master/dl_emotion_recognition_uea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1><strong>Dataset: Natural Human Face Images for Emotion Recognition</strong></h1>
    <h4>Link: <a href="https://www.kaggle.com/sudarshanvaidya/random-images-for-face-emotion-recognition">sa</a></h4>
    <div id="text">
        <p>
São 5.500 + imagens com 8 categorias de emoções - raiva, desprezo, nojo, medo, felicidade, neutralidade, tristeza e surpresa. Todas as imagens contêm rosto humano em tons de cinza (ou esboço). Cada imagem tem uma escala de cinza de 224 x 224 pixels no formato PNG.</p>
<p>As imagens são obtidas na Internet, onde estão disponíveis gratuitamente para download. Google, Unsplash, Flickr etc.</p>
    </div>
</div>

In [ ]:
import os
import cv2
import math
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, GlobalAvgPool2D, GlobalMaxPool2D
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

from keras.utils import np_utils

In [ ]:
# Currenty Directory
PATH_ROOT = !pwd
PATH_ROOT

# Global Vars
DATASET_FILE = 'dl-emotion-recognition-uea.zip'
DATASET_GOOGLE_DRIVER_ID = '1J0xMHoQqC6zBibSxukK-Gshfb5kK-P1y'
DATASET_PATH = PATH_ROOT[0] + '/dataset/'

# Download Dataset.zip from GoogleDriver
!gdown --id $DATASET_GOOGLE_DRIVER_ID
!unzip -o $DATASET_FILE

# !ls -la $DATASET_PATH

# DATASET_PATH = '/home/charles/uea/deep-learning/datasets/dl-emotion-recognition-uea/'
# !ls -la $DATASET_PATH

In [ ]:
!ls -la $DATASET_PATH

In [ ]:
# Processing directory dataset

total_images = 0
classes = []
for _dir in os.listdir(DATASET_PATH):
    count = 0
    for f in os.listdir(DATASET_PATH + _dir + "/"):
        count += 1
        total_images += 1
        
    classes.append(_dir)
    print(f"{_dir} => {count} imagens")
    
print('\n----------------------------------------------')    
print(f"\nTotal: {total_images} imagens")
print('Total de classes: ', len(classes))
print('Classes: ', classes)
print('\n----------------------------------------------')    

In [ ]:
img_arr = np.empty(shape=(total_images, 224, 224, 3))
img_label = np.empty(shape=(total_images))
label_to_text = {}

i = 0
e = 0
for dir_ in os.listdir(DATASET_PATH):
    if dir_ in classes:
        label_to_text[e] = dir_
        for f in os.listdir(DATASET_PATH + dir_ + "/"):
            img_arr[i] = cv2.imread(DATASET_PATH + dir_ + "/" + f)
            img_label[i] = e
            i += 1
        print(f"loaded all {dir_} images to numpy arrays")
        e += 1

img_arr.shape, img_label

In [ ]:
print(img_arr[-1])

In [ ]:
# Classes

label_to_text

In [ ]:
# Visualização Randômica de 2 imagens por classe.

fig = pyplot.figure(1, (8, 8))

idx = 0
for k in label_to_text:
    
    sample_indices = np.random.choice(np.where(img_label==k)[0], size=2, replace=False)
    sample_images = img_arr[sample_indices]
    
    for img in sample_images:
        idx += 1
        ax = pyplot.subplot(4, 4, idx)
        ax.imshow(img[:,:,0], cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(label_to_text[k])
        pyplot.tight_layout()


In [ ]:
# X, Y = np.array(img_arr) / 255.0, np.array(img_label)
X = np.array(img_arr) / 255
X

In [ ]:
Y = np.array(img_label)
Y

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [ ]:
# Converter para valores multiclasse. Valores convertidos para uma matriz que vão categorizar os dados
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
y_train, y_test

In [ ]:
num_classes = y_test.shape[1]

In [ ]:
# Model

model = Sequential()

model.add(Conv2D(30, (4, 4), input_shape=(6, 224, 244), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dense(num_classes, activation='softmax', name='Predict'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
# Train

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=7, batch_size=32)